# Administer your Workforce Project with Python

Welcome to Jupyter Notebooks. This is a useful tool that can be used to manage your ArcGIS Online Organization through the use of Python. In this notebook we take existing hosted feature layers in ArcGIS Online and integrate it with a Workforce project to make assignments.

In [1]:
from datetime import datetime
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps import workforce
from getpass import getpass

Start by connecting to your GIS Organization. Provide a username, password and URL.

In [2]:
password = getpass()

········


In [3]:
gis = GIS("https://esrican-desktop.maps.arcgis.com/home/organization.html", "goconnell_desktop", password)

Connect to your existing workforce project in your ArcGIS Online organization.

In [4]:
item = gis.content.get("b70aaefc86654858aee5671d591b86cd")
project = workforce.Project(item)

Determine the assignment type you will be creating and the dispatcher of the project. Identify the layer that has the point data you will be using to create assignments. 

In [5]:
patrol_division = project.assignment_types.get(name="Patrol Division")
dispatcher = project.dispatchers.get(user_id="goconnell_desktop")

In [16]:
layer = gis.content.get("db86949572404bc6a7125b189897075d").layers[0]
patrols = layer.query("1=1", out_sr=3857).features

Use the map widget to examine your data - how many events are there, what types are they, and will they need further parsing before becoming assignments?

In [17]:
patrols_map = gis.map("Naperville, IL", zoomlevel=12)
patrols_map.add_layer(layer)
patrols_map

MapView(layout=Layout(height='400px', width='100%'), zoom=12.0)

Append your features to your assignments layer and field map all the required data.

In [18]:
assignments = []
for feature in patrols:
    assignments.append(
        workforce.Assignment(
            project,
            geometry=feature.geometry,
            location=feature.attributes["location"],
            description=feature.attributes["activity_observed"],
            priority=feature.attributes["priority"],
            assignment_type=patrol_division,
            dispatcher=dispatcher,
            status="unassigned"
        )
    )
project.assignments.batch_add(assignments)

[<Assignment 31>,
 <Assignment 32>,
 <Assignment 33>,
 <Assignment 34>,
 <Assignment 35>,
 <Assignment 36>,
 <Assignment 37>,
 <Assignment 38>,
 <Assignment 39>,
 <Assignment 40>,
 <Assignment 41>,
 <Assignment 42>,
 <Assignment 43>,
 <Assignment 44>,
 <Assignment 45>]

QA your results, is your assignment list complete?

In [20]:
proj_assignments = project.assignments.search()
proj_assignments

[<Assignment 31>,
 <Assignment 32>,
 <Assignment 33>,
 <Assignment 34>,
 <Assignment 35>,
 <Assignment 36>,
 <Assignment 37>,
 <Assignment 38>,
 <Assignment 39>,
 <Assignment 40>,
 <Assignment 41>,
 <Assignment 42>,
 <Assignment 43>,
 <Assignment 44>,
 <Assignment 45>]

All of your assignments we added to the Patrol Division by default. The following code block parses through the assignments, and reassigns the types as needed.

In [15]:
# List all assignment types
assignment_types = project.assignment_types.search()
for at in assignment_types:
    print(f"Type: {at.name}")

Type: Site Visit
Type: Patrol Division
Type: Community Policing
Type: Narcotics Unit
Type: Major Crimes Unit
Type: Office of the Mayor


In [27]:
for a in proj_assignments:
    if a.description == "DrugParaphernalia":
        a.assignment_type = "Site Visit"
    elif a.description == "DrugSale":
        a.assignment_type = "Major Crimes Unit"
    elif a.description == "PatientFraud" or "PrescriptionFraud":
        a.assignment_type = "Community Policing"
    elif a.description == "DrugTheft":
        a.assignment_type = "Narcotics Unit"
    else:
        a.assignment_type = "Patrol Division"

In [28]:
for a in proj_assignments:
    print(f"Type: {a.assignment_type}")

Type: Site Visit
Type: Major Crimes Unit
Type: Site Visit
Type: Community Policing
Type: Community Policing
Type: Community Policing
Type: Community Policing
Type: Major Crimes Unit
Type: Community Policing
Type: Community Policing
Type: Site Visit
Type: Community Policing
Type: Community Policing
Type: Site Visit
Type: Site Visit
